In [2]:
pip install sentence_transformers 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.1 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 17.7 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install langchain_core langchain_chroma langchain_text_splitters langchain langchain_google_genai sentence_transformers langsmith

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 801.7 kB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 4.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 17.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import pandas as pd
import numpy as np
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.schema import Document
from sentence_transformers import SentenceTransformer , util
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.documents import Document
from kaggle_secrets import UserSecretsClient
from langchain.load import dumps, loads
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import faiss

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
import os

# List all files in the directory
directory_path = '/kaggle/input/sanad-dataset'
files = os.listdir(directory_path)
print(files)


['Finance', 'Politics', 'Religion', 'Sports', 'Medical', 'Culture', 'Tech']


In [8]:
import os
import pandas as pd

def load_data_from_folders(main_folder_path):
    data = []

    # المرور على كل مجلد فرعي
    for category_folder in os.listdir(main_folder_path):
        category_path = os.path.join(main_folder_path, category_folder)

        # التحقق من أن المسار هو مجلد
        if os.path.isdir(category_path):
            category_label = category_folder  # اسم المجلد هو اسم الفئة

            # المرور على كل ملف داخل المجلد الفرعي
            for file_name in os.listdir(category_path):
                file_path = os.path.join(category_path, file_name)

                # التأكد من أن المسار هو ملف وليس شيء آخر
                if os.path.isfile(file_path):
                    with open(file_path, 'r', encoding='utf-8') as file:
                        content = file.read()
                        data.append({'text': content, 'category': category_label})
    
    # تحويل البيانات المجمعة إلى DataFrame
    return pd.DataFrame(data)

# المسار إلى المجلد الرئيسي الذي يحتوي على المجلدات الفرعية
main_folder_path = '/kaggle/input/sanad-dataset'

# تحميل البيانات من المجلدات
WholeData = load_data_from_folders(main_folder_path)


In [9]:
WholeData.head()

,text,category
0,"دبي - ""الخليج"":حصدت شعاع كابيتال جائزة ""أفضل ش...",Finance
1,أبوظبي - علي أسعد: تراجعت أسواق المال في الدول...,Finance
2,استأنفت أسواق الأسهم المحلية أمس تحركها باتجاه...,Finance
3,دبي «الخليج»: أعلنت شركة تكافل الإمارات عن بدء...,Finance
4,تحتفل شركة طاقة الخليج البحرية، التي تتخذ من د...,Finance


In [10]:
# Preprocessing the text data (e.g., remove NaNs, tokenize, etc.)
WholeData.dropna(subset=['text'], inplace=True)
# استخراج النصوص من إطار البيانات
documents = WholeData['text'].tolist()


In [11]:
# Initialize the SentenceTransformer model for embedding
model = SentenceTransformer('all-mpnet-base-v2')

# Encode the documents
document_embeddings = model.encode(documents, convert_to_tensor=True)

# Verify shape and type of embeddings
print(document_embeddings.shape)


# إنشاء مؤشر FAISS
index = faiss.IndexFlatL2(document_embeddings.shape[1])

# إضافة التضمينات النصية إلى المؤشر
index.add(document_embeddings.cpu().numpy())



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1422 [00:00<?, ?it/s]

torch.Size([45500, 768])


In [12]:
def retrieve(query, model, index, documents, k=5):
    # تضمين الاستعلام
    query_embedding = model.encode(query, convert_to_tensor=True)
    
    # إجراء البحث عن التشابه
    D, I = index.search(query_embedding.cpu().numpy(), k)
    
    # استرجاع وإرجاع أفضل k مستندات
    results = [documents[i] for i in I[0]]
    return results


In [20]:
from transformers import BertTokenizer, BertModel
import torch

# تحميل نموذج BERT والمحول
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def embed_text_bert(text):
    # ترميز النص باستخدام المحول الخاص بـ BERT
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    
    # تمرير المدخلات إلى نموذج BERT للحصول على التضمينات
    with torch.no_grad():
        outputs = bert_model(**inputs)
    
    # استخدام آخر طبقة مخفية كتمثيل نص
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

def retrieve(query, model, index, documents, k=5):
    # تضمين الاستعلام باستخدام BERT
    query_embedding = embed_text_bert(query)
    
    # تحويل التضمين إلى numpy للبحث في FAISS
    query_embedding_np = query_embedding.cpu().numpy()
    
    # إجراء البحث عن التشابه
    D, I = index.search(query_embedding_np, k)
    
    # استخراج المستندات بناءً على النتائج
    retrieved_docs = [documents[i] for i in I[0]]
    return retrieved_docs


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [21]:
query = "Your customer query here"

# استرجاع أفضل المستندات
retrieved_docs = retrieve(query, model, index, documents)

# استخدام نموذج توليدي مثل T5 لتكوين الرد
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model_t5 = T5ForConditionalGeneration.from_pretrained('t5-small')

# تحضير النص المدخل من خلال دمج المستندات المسترجعة
input_text = " ".join(retrieved_docs)
input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True)

# توليد الرد
outputs = model_t5.generate(input_ids, max_length=150)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [23]:
import numpy as np
from sentence_transformers import SentenceTransformer

# استخدم نموذج التضمين نفسه
model = SentenceTransformer('all-mpnet-base-v2')

def embed_documents(documents):
    """تضمين المستندات"""
    return model.encode(documents, convert_to_tensor=True)

def retrieve_documents(query, index, documents, k=3):
    """استرجاع أفضل المستندات"""
    query_embedding = embed_documents([query])[0]
    
    # التأكد من أن التضمين لديه الأبعاد الصحيحة
    query_embedding_np = query_embedding.cpu().numpy().reshape(1, -1)
    
    # استرجاع أقرب k مستندات
    distances, indices = index.search(query_embedding_np, k)
    retrieved_docs = [documents[i] for i in indices[0]]
    return retrieved_docs

def generate_response(query, retrieved_docs):
    """توليد الرد باستخدام المستندات المسترجعة"""
    # دمج النصوص المسترجعة لتوليد الرد
    input_text = " ".join(retrieved_docs)
    
    # استخدام نموذج توليدي (مثل T5) لتوليد الرد
    tokenizer = T5Tokenizer.from_pretrained('t5-small')
    model_t5 = T5ForConditionalGeneration.from_pretrained('t5-small')
    
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True)
    outputs = model_t5.generate(input_ids, max_length=150)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return response

# استرجاع وتوليد الرد
retrieved_docs = retrieve_documents(query, index, documents)
response = generate_response(query, retrieved_docs)

print("Response:", response)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Response: 
